In [34]:
from typing import List
import os
import random
import json
import re
from joblib import dump, load

from gensim.summarization.bm25 import BM25
import jieba
from tqdm import tqdm
from scipy.linalg import norm

In [2]:
data_path = "sohu2021_open_data_clean/"

In [3]:
label_type_to_id = {'labelA':0, 'labelB':1}
label_to_id = {'0':0, '1':1}

In [4]:
def get_text_iterator(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            yield line

In [5]:
def _transform_text(text):
   text = text.strip().replace('\n', '。').replace('\t', '').replace('\u3000', '')
   return re.sub(r'。+', '。', text)

In [6]:
def get_data_iterator(data_path, file_names):
    # TODO: 随机取
    file_iters = []
    for file_name in file_names:
      for category in os.listdir(data_path):
          category_path = os.path.join(data_path, category)
          if not os.path.isdir(category_path):
              continue
              
          file_path = os.path.join(category_path, file_name)
          if not os.path.isfile(file_path):
              continue
              
          
          file_iter = get_text_iterator(file_path)
          cat_source = 0
          if category[0] == '长':
            cat_source = 1
          cat_target = 0
          if category[1] == '长':
            cat_target = 1
          file_iters.append((file_iter, cat_source, cat_target))
        
    while len(file_iters) > 0:
        i = random.randrange(len(file_iters))
        line = next(file_iters[i][0], None)
        cat_source = file_iters[i][1]
        cat_target = file_iters[i][2]
        if line is None:
            del file_iters[i]
            continue
            
        data = json.loads(line)

        data['source'] = _transform_text(data['source'])
        if len(data['source']) == 0:
            print('source:', line, data)
            break
#                     continue

        data['target'] = _transform_text(data['target'])
        if len(data['target']) == 0:
            print('target:', line, data)
            break
#                     continue

        label_name_list = list(key for key in data.keys() if key[:5]=='label')
        if len(label_name_list) != 1:
            print('label_name_list:', line, data)
            break
#                     continue
        label_name = label_name_list[0]
        if data[label_name] not in label_to_id.keys():
            print('label_name:', line, data, label_name)
            break
#                     continue
        
        label_dict = {key: -1 for key in label_type_to_id.keys()}
        label_dict[label_name] = label_to_id[data[label_name]]
        if label_dict['labelA'] == 0:
            label_dict['labelB'] = 0
        if label_dict['labelB'] == 1:
            label_dict['labelA'] = 1

        yield data['source'], data['target'], cat_source, cat_target, label_dict['labelA'], label_dict['labelB']

In [7]:
it = get_data_iterator(data_path, ["train.txt", "valid.txt", "round2.txt"])

In [8]:
next(it)

('48岁吴绮莉现身避谈小龙女，主动提生父，对方墓地买好不愿回港 48岁吴绮莉现身避谈小龙女，主动提生父，对方墓地买好不愿回港 重新回到香港生活的吴绮莉，虽然时不时就会因为小龙女吴卓林的事情上头条，但是这几年慢慢回归事业的她，状态倒是越来越好了。曾经她也说过女儿已经成年了，自己也应该有自己的生活了。  48岁的吴绮莉现身某电视台，为自己即将上映的新片做宣传，当天吴绮莉的造型也是非常的保暖，一身长款的黑衣外套，看起来气色不错。虽然刚带着小龙女回港的那几年，吴绮莉也是非常沧桑，但是经过这几年的保养，状态已经今时不同往日了，毕竟她也曾是港姐冠军，底子还是很好的。  吴绮莉这次宣传的新剧，正是她去年的时候参与拍摄的《太平纹身店》。这部剧杀青也是大半年了，主要还是因为疫情的影响，所以才会一直拖到现在，才开始宣传和上映，这也是吴绮莉复出后的首部影视作品，因此她也是非常看重！  事实上，港姐出身的吴绮莉曾经也像大多数港姐一样，签约了TVB成为演员，后来生下小龙女之后，她除了《雷霆扫军》就没有其他作品了，在离开TVB之后更是选择在电台工作。  去年5月份的时候，吴绮莉选择再次回归演员的身份，但合作的却不是TVB，而且老东家的对手台某TV，拍摄的正是这部《太平纹身店》，虽然吴绮莉已经有七年没有拍过戏了，但她也曾经是一名专业的演员，可以期待一下。  不过相比吴绮莉的新作品，还是她和小龙女的事情更受媒体的关注，我们都知道在小龙女的成长过程中，和吴绮莉的母女关系也是发生很多的变化，特别有了交往和结婚对象之后，母女两的关系更是越来越差。  这次媒体也问起了吴绮莉和小龙女目前的情况，但是吴绮莉表示不想谈和女儿的私事，甚至为了避免谈小龙女的事情，主动提起了自己生父的事情。  前段时间，吴绮莉突然在自己的社交账号上自曝，称已经失联37年的父亲突然给她打了电话，让她又惊又喜。据吴绮莉的描述，从她记事起，这辈子只见过父亲两次，一次是妈妈离婚带走自己的时候，还有一次则是11岁的时候，在街头匆匆的偶遇了一回。  看得出这次久违的联系，父女两的对话也是让人动容，吴绮莉更是没有因为多年不联系而觉得陌生或者排斥，反而有点珍惜这份来之不易的“父爱”。吴绮莉在活动中说道，父亲定居美国多年，但如今身边的亲朋好友们都已经搬走了，只剩下他一个人独居，所以吴绮莉有打算将八十来岁的老父亲接回香港照顾。  只是吴绮莉的父亲似乎

In [9]:
def get_seg_iterator(data_path, file_names):
    data_it = get_data_iterator(data_path, file_names)
    for source, target, _, _, labelA, labelB in data_it:
        label = 0
        if labelA == 1 or labelB == 1:
            label = 1
        yield jieba.lcut(source), jieba.lcut(target), label

In [10]:
it = get_seg_iterator(data_path, ["train.txt", "valid.txt", "round2.txt"])

In [11]:
next(it)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.680 seconds.
Prefix dict has been built successfully.


(['马',
  '卡比',
  '获得',
  '了',
  '胜利',
  '，',
  '并且',
  '取得',
  '了',
  '击败',
  '巴斯',
  '科尼亚',
  '的',
  '决胜局'],
 ['今天',
  '是',
  'NBA',
  '一个',
  '大',
  '的',
  '比赛',
  '日',
  '，',
  '联盟',
  '今天',
  '的',
  '比赛',
  '比较',
  '多',
  '，',
  '虽然',
  '没有',
  '多少',
  '强强',
  '对话',
  '的',
  '焦点',
  '赛事',
  '，',
  '但是',
  '也',
  '打出',
  '了',
  '一些',
  '值得',
  '球迷',
  '们',
  '回味',
  '的',
  '话题',
  '。',
  '比如',
  '今天',
  '的',
  '比赛',
  '里',
  '，',
  '骑士',
  '中锋',
  '德',
  '拉蒙德',
  '砍',
  '下',
  '33',
  '分',
  '23',
  '篮板',
  '的',
  '数据',
  '，',
  '生涯',
  '第四次',
  '拿到',
  '30',
  '+',
  '20',
  '的',
  '数据',
  '，',
  '他',
  '成为',
  '了',
  '今天',
  '的',
  '焦点',
  '之一',
  '；',
  '另外',
  '，',
  '公牛',
  '球星',
  '拉文',
  '在',
  '今天',
  '砍',
  '下',
  '35',
  '分',
  '7',
  '篮板',
  '6',
  '助攻',
  '，',
  '连续',
  '3',
  '场',
  '比赛',
  '砍',
  '下',
  '至少',
  '35',
  '分',
  '5',
  '篮板',
  '5',
  '助攻',
  '，',
  '队史',
  '上',
  '追平',
  '了',
  '了',
  '迈克尔',
  '-',
  '乔丹',
  '之前',
  '创造',
  '的',
  '队史',
  '记录',
  '。',
  '

In [12]:
def load_stopwords_from_file(stopwords_file:str=None) -> set:
    if stopwords_file is None:
        return set()
    if not os.path.exists(stopwords_file):
        raise ValueError("stopwords_file: {} doesn't not exist".format(stopwords_file))

    stopwords = set()
    with open(stopwords_file,'r') as fr:
        lines = fr.readlines()
        for line in lines:
            word = line.strip()
            stopwords.add(word)
    return stopwords

In [13]:
stopwords = load_stopwords_from_file("stopwords/hit_stopwords.txt")

In [14]:
# stopwords

In [15]:
def train_bm25Model(data_path, file_names, Stopwords: set) -> BM25:
    seg_it = get_seg_iterator(data_path, file_names)
    corpus = []
    for seg_source, seg_target, _ in tqdm(seg_it):
        corpus.append([word for word in seg_source if word not in Stopwords])
        corpus.append([word for word in seg_target if word not in Stopwords])
    bm25Model = BM25(corpus)
    return bm25Model

In [45]:
# bm25Model = train_bm25Model(data_path, ["train.txt", "valid.txt", "round2.txt"], stopwords)

119168it [10:08, 195.95it/s]


In [46]:
# bm25Model

In [49]:
# dump(bm25Model, "bm25.bin")

['bm25.bin']

In [50]:
# bm25Model.idf

{'春节': 2.7614518530715735,
 '档': 3.7612473432229265,
 '据说': 4.806208258771681,
 '刺杀': 5.081210180800156,
 '小说家': 5.105430622320494,
 '全宇宙': 10.030021604104856,
 '最好': 3.208635071074026,
 '剧组': 4.393073875102055,
 '两个': 2.357970268761248,
 '世界': 2.7606266582015646,
 '一场': 2.4819626372021144,
 '奇幻': 5.294304706678292,
 '冒险': 5.268467752479191,
 '赵继伟': 5.748234045903537,
 '单节': 4.903539040821128,
 '15': 2.2883264216843937,
 '分': 1.9320809330864144,
 '26': 3.0362373535368494,
 '10': 1.7264320457893092,
 '极限': 5.609867717193497,
 '拯救': 5.438334287056515,
 '辽宁': 4.2827053785733025,
 '郭': 4.3494909482790955,
 '艾伦': 4.992237961958343,
 '才': 2.0735242322873386,
 '在场': 4.859736298447572,
 '梦游': 7.801179371187857,
 '5': 1.5215057314377294,
 '战': 4.484706534169692,
 '4': 1.4673753671107512,
 '败': 6.462332120005181,
 'CBA': 3.9863783862648727,
 '夺冠': 3.8742529356339617,
 '热门': 4.218526832120515,
 '崩盘': 6.0664824470764165,
 '超级': 3.6758415377915306,
 '外援': 4.079019102780432,
 '组合': 3.800513058498742

In [16]:
bm25Model = load("bm25.bin")

In [18]:
def calculate_bm25_similarity(bm25Model: BM25, tokens1: List[str], tokens2: List[str]) -> float:
    token_sequence1, token_sequence2 = (tokens1, tokens2) if len(tokens1) > len(tokens2) else (tokens2, tokens1)

    word_sequences = {}
    # todo: 为什么只计算token_sequence2的词频
    for word in token_sequence2:
        if word not in word_sequences:
            word_sequences[word] = 0
        word_sequences[word] += 1

    param_k1 = 1.5
    param_b = 0.75

    score1, score2 = 0.0, 0.0
    for word in token_sequence1:
        if word not in word_sequences or word not in bm25Model.idf:
            continue
        score1 += (bm25Model.idf[word] * word_sequences[word] * (param_k1 + 1) / (
                word_sequences[word] + param_k1 * (1 - param_b + param_b * 1)))
    for word in token_sequence2:
        if word not in word_sequences or word not in bm25Model.idf:
            continue
        score2 += (bm25Model.idf[word] * word_sequences[word] * (param_k1 + 1) / (
                word_sequences[word] + param_k1 * (1 - param_b + param_b * 1)))

    similarity = score1 / score2 if score2 > 0 else 0
    similarity = min(similarity, 1.0)
    return similarity

In [19]:
it = get_seg_iterator(data_path, ["train.txt", "valid.txt", "round2.txt"])

In [33]:
data = next(it)
"".join(data[0]), "".join(data[1]), data[2], calculate_bm25_similarity(bm25Model, data[0], data[1])

('文|汤汤慢元宵佳节，电视剧《锦心似玉》又悄悄上映了，匪夷所思的是，这部剧由于主演是钟汉良和谭松韵一直被网友看好，又改编自当红小说《淑女攻略》，竟然未能上星在卫视播出，着实让人奇怪。',
 'Hello，大家好！今天的你们都开心了吗？都在忙些什么呢？今天要和大家一起聊聊近期热度较大的三部网剧豆瓣最新成绩单，瞅瞅你正在追的是哪一部呢？ ●张哲瀚、龚俊《山河令》12w人参与评分，综合分数8.4  《山河令》豆瓣8.4分！太牛了吧！上架十天，12万人打分，好牛，竟然还能走高，又一爆剧！这剧尺度有点大，fu女的狂欢，编剧也是业内良心的编剧了，两个小糊咖都脱糊了！特别是龚俊要爆成流量了。  《山河令》现在的豆瓣评分8.4分，还有要上涨的趋势。《山河令》与金庸古龙的武侠剧最大的不同是它融入了现代的元素。剧集里的台词中有很多的古诗词，编剧还加入了一些现代的流行语，比如舔狗、小姐姐等词语。而张哲瀚和龚俊也因为该剧圈粉无数。龚俊的粉丝从865万涨至930万，短短数日狂涨粉丝近50多万。张哲瀚的粉丝数也大涨了近27万。两位主演的演技让很多人赞叹，打戏也很出彩。一招一式毫不敷衍，动作干净利落，很有舞蹈的美感。与龚俊和张哲瀚在《山河令》中一起仗剑江湖吧！  ●郭麒麟、宋轶《赘婿》13w人参与评分，综合分数6.9  《赘婿》播出至今热度很高，观众口碑却始终上不去，目前豆瓣只有6.9分，远低于预期，也不及制作团队此前作品《庆余年》的分数。很多网友都表示对这部剧失望了，从期望到失望的落差有点大，但的确是翻车了。  究竟问题出在哪？很多人都表示郭德纲之子星二代郭麒麟长相太路人，不适合做男主角，影响了观感，这当然也算是问题之一。但最大的问题出在剧本上，编剧要背锅，对原著的改编太大才是剧集口碑翻车的真正原因。《赘婿》改编自“愤怒的香蕉”的同名网络小说，讲述一个男人以现代金融界巨头的身份穿越到古代，进入一个商贾之家成为赘婿之后，如何逆袭并娶到七个老婆的“网文”式小说。而剧集将“七个老婆”的故事改掉了，变成了履行一夫一妻制，宣扬男女平等的正能量剧。这样改虽然符合大众审美，却遭遇原著粉的疯狂抵制，而女性观众了解原著之后，也对剧集提不起兴趣，可以说是两面不讨好。  ●钟汉良、谭松韵《锦心似玉》2w多人参与评分，综合分数6.5  《锦心似玉》豆瓣开分6.5分！由钟汉良谭松韵主演的《锦心似玉》豆瓣开分了，这部剧

In [35]:
def gen_tf(tokens:List[str]) -> dict:
    '''
    TF(term frequency) is defined as:
        tf(token) = Frequency(token) / Count(all_tokens)
    :param tokens: List[str]
    :return: Dict
    '''
    total = len(tokens)
    tf_dict = {}
    for word in tokens:
        tf_dict[word] = tf_dict.get(word, 0.0) + 1.0
    for word in tf_dict:
        tf_dict[word] /= total
    return tf_dict


def gen_tfidf(tokens:List[str], idf_dict:dict) -> dict:
    '''
    Tf-Idf(term frequency–inverse document frequency) is defined as:
        tf(token) = Frequency(token) / Count(all_tokens)
        idf(token) is implemented by querying the bm25 model, whose building function is:
            idf(token) = log( (Count(all_docs) - Count(contain_token_docs) + 0.5) / (Count(contain_token_docs) + 0.5) )
        tf-idf(token) = tf(token) * idf(token)
    :param tokens: List[str]
    :param idf_dict: Dict[(str,float)]
    :return:
    '''
    total = len(tokens)
    tfidf_dict = {}
    for w in tokens:
        tfidf_dict[w] = tfidf_dict.get(w, 0.0) + 1.0
    for k in tfidf_dict:
        tfidf_dict[k] *= idf_dict.get(k, 0.0) / total
    return tfidf_dict


def cosine_similarity(a:dict, b:dict) -> float:
    longer_dict, shorter_dict = (a, b) if (len(a) > len(b)) else (b, a)
    res = 0
    for (word_name, word_value) in longer_dict.items():
        res += word_value * shorter_dict.get(word_name, 0)
    if res == 0:
        return 0.
    try:
        res = res / (norm(list(longer_dict.values())) * norm(list(shorter_dict.values())))
    except ZeroDivisionError:
        res = 0.
    return res

def calculate_tf_cosine_similarity(tokens1:List[str], tokens2:List[str]) -> float:
    tf1 = gen_tf(tokens1)
    tf2 = gen_tf(tokens2)
    return cosine_similarity(tf1, tf2)


def calculate_tfidf_cosine_similarity(tokens1:List[str], tokens2:List[str], idf_dict:dict) -> float:
    tfidf1 = gen_tfidf(tokens1, idf_dict)
    tfidf2 = gen_tfidf(tokens2, idf_dict)
    return cosine_similarity(tfidf1, tfidf2)

In [41]:
data = next(it)
"".join(data[0]), "".join(data[1]), data[2], calculate_bm25_similarity(bm25Model, data[0], data[1]), calculate_tf_cosine_similarity(data[0], data[1]), calculate_tfidf_cosine_similarity(data[0], data[1], bm25Model.idf)

('👆点击进入珠海通小程序，免费发布便民信息 《巴啦啦小魔仙》相信很多人都看过， 但今天下午传来了一个不幸的消息， 凌美琪扮演者孙侨潞已经离开了我们！ 年仅25岁！死因疑似是心梗猝死… (R.I.P.    )   消息刚出，让网友们震惊不已， 纷纷表示"姐姐，求你出来辟谣啊" 还有不少网友则直呼 "希望姐姐好好的""希望不是真的"。  不过随后， 孙侨潞妈妈用她的账号发文确认了 #孙侨潞去世#的消息。  孙侨潞妈妈还表示： "我在这里恳求大家，不要做过多的揣测和评价， 逝者已逝，谢谢大家了。"   孙侨潞和妈妈合影 如今才25岁的孙侨潞 因在《巴拉拉小魔仙》中 饰演凌美琪魔仙一角而爆火， 收获了无数粉丝…  但是长大后的孙侨潞发展一般， 也逐渐的淡出了娱乐圈， 这让她慢慢的被观众淡忘…  得知该消息后的网友 不敢相信 这位存在于大家童年记忆中的女生 去世得如此突然…  正如孙侨潞的母亲所言， 是魔仙堡需要她，对于孙侨潞的离去， 我们表示悲痛和不舍， 而孙侨潞永远是我们心中的小美琪。 也希望她一路走好，愿天堂没有伤痛… R.I.P.     什么是心肌梗死？ 心梗，近年来，这个危险的心脏疾病屡屡与死亡联系在一起。 在心肌梗死的"死亡名单"上囊括了年轻人、中年人、老年人，运动员、商界精英、政要、演员导演、医护人员等赫然在列，更有无数普通人。 而且，很多人此前并没有心脏病史。 心肌梗死，简单说就是心脏的动脉血管，因为血管斑块破裂形成血栓，血栓堵塞了血管，导致血管失去血流，从而引起心脏肌肉坏死，引起的一种心脏病。 心肌梗死不但是引起猝死的主要原因，如果不积极正规治疗还会引起心衰，一旦发生心衰，5年死亡率是50%。也就是心肌梗死很严重，不但会猝死，还会缩短寿命。 都说人在中青年阶段应该身体康健才对 然而"心梗、中风这些以往的"老年病 却越来越把中年轻人写上了"死亡名单"      归根到底， 这和当下很多人不良的生活习惯 有着重要关系！！！ 熬夜、作息不规律、大量抽烟、喝酒 饮食习惯不好，如过度饱餐或高脂饮食… 这些都可能是诱发心梗的因素！  男女心梗发作信号不一样 男持续胸痛和大汗 男性急性心梗发作症状很明确，主要是持续胸痛和大汗; 女放射痛和恶心呕吐 女性患者则主要是放射痛、后背痛、恶心呕吐多，胸痛和大汗相对少见，有时"自己都说不清楚"。 虽然急性心梗时，胸痛会比较普遍

In [1]:
from my_utils import calculate_bm25_similarity, calculate_tf_cosine_similarity, calculate_tfidf_cosine_similarity